# Super-structure scenario modelling in brightway

This notebook shows how to use the bw_scenarios module, using the example of a bike.

We start by importing an SDF file (.xslx or .csv).

## Project setup

In [1]:
import bw2data as bd
import bw2io as bi

In [2]:
PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"
print("Check")

Check


In [3]:
# this is the current preferred way to install a biosphere database w/bw25
# it will create biosphere named "ecoinvent-3.10-biosphere"
# and add the associated LCIA methods

if PROJECT_NAME in bd.projects:
    pass
else:
    bi.remote.install_project(
    "ecoinvent-3.10-biosphere", PROJECT_NAME, overwrite_existing=True
)

In [4]:
bd.projects.set_current(PROJECT_NAME)

In [5]:
bd.databases

Databases dictionary with 2 object(s):
	bike_production_example
	ecoinvent-3.10-biosphere

In [5]:
xl_importer = bi.importers.ExcelImporter("bike_production_example.xlsx")

Extracted 1 worksheets in 1.22 seconds


In [7]:
# If there are unlinked exchanges because of biosphere name change,
# use the right biosphere name

In [6]:
for data_item in xl_importer.data:
    for exchange in data_item["exchanges"]:
        if exchange["type"] == "biosphere" and exchange["database"] == "biosphere3":
            print(f"Update bio db name for {exchange}")
            exchange["database"] = BIO_DB_NAME

In [7]:
xl_importer.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 14.35 seconds


In [8]:
xl_importer.statistics()

4718 datasets
	0 exchanges
	0 unlinked exchanges (0 unique)
	


(4718, 0, 0, 0)

In [9]:
if xl_importer.statistics(print_stats=False)[3] == 0:
    xl_importer.write_database()

100%|██████████| 4718/4718 [00:00<00:00, 8559.43it/s]


Vacuuming database 
Created database: biosphere3


In [10]:
bd.databases

Databases dictionary with 3 object(s):
	bike_production_example
	biosphere3
	ecoinvent-3.10-biosphere

In [11]:
bike_db = bd.Database("bike_production_example")

In [12]:
for a in bike_db:
    print(f"{a} -> {a['code']}")

'carbon fibre production' (kg, DE, None) -> cf
'bike production' (bike, DK, None) -> bike
'natural gas production' (MJ, NO, None) -> ng


## Extract the data from the SDF file

In [13]:
import sys
sys.path.append("../")
from pathlib import Path

from bw_scenarios.importer import SDFImporter

file_path = Path("sdf.xlsx")

# store the scenario data in a variable
scenarios = SDFImporter.from_excel(file_path)

In the .sdf-file it is defined how the activities are changing within the distinct scenarios.

The file used for this example is specified above.

In the graphic below, the direction of change per scenario and exchange is visualized.

![scenarios_visualized.png](scenarios_visualized.png)